<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Carlos_Ancasi/Aula_8_Exercicio_Carlos_Ancasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'
print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 29.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 21:41:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokensIds_n = []
        self.y = []
        for text in tqdm_notebook(texts):
            tokens_ids = tokenize(text, tokenizer)
            if len(tokens_ids) < context_size + 1:
                continue
            for i in range(len(tokens_ids)-context_size):
                self.tokensIds_n.append(tokens_ids[i:i+context_size])      
                self.y.append(tokens_ids[i+context_size])
                
    def __len__(self):  
        return len(self.tokensIds_n)

    def __getitem__(self, idx):
        
        return torch.tensor(self.tokensIds_n[idx]).long(), torch.tensor(self.y[idx]).long()

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 21:41:34--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 173.194.196.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   183MB/s    in 0.6s    

2022-05-25 21:41:35 (183 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [9]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [16]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super().__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # C()
        self.C_w = nn.Embedding(vocab_size, embedding_dim)

        # P()
        self.P_w = nn.Embedding(context_size, embedding_dim)

        self.K_w = nn.Linear(embedding_dim, embedding_dim , bias=False)
        self.Q_w = nn.Linear(embedding_dim, embedding_dim , bias=False)
        self.V_w = nn.Linear(embedding_dim, embedding_dim , bias=False)
        self.E_w = nn.Linear(embedding_dim, embedding_dim , bias=False)

        hidden_size = 2*embedding_dim
        self.linear1 = nn.Linear(embedding_dim, hidden_size)
        self.relu1 = nn.ReLU()
        # self.linear2 = nn.Linear(hidden_size, hidden_size)

        # self.downsample = nn.Conv1d(1, 1, kernel_size=1, stride=1, padding=0, groups=1, bias=False, dilation=1)

        # self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_size, vocab_size, bias=False)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        C_emb = self.C_w(inputs)
        P_emb = self.P_w(torch.LongTensor(range(0,self.context_size)).to(inputs.device)).unsqueeze(0)

        X = C_emb + P_emb
        X_end = X[:,-1,:].unsqueeze(1)
        Q = self.Q_w(X_end)
        K = self.K_w(X)
        V = self.V_w(X)

        scores = torch.matmul(Q, torch.transpose(K,1,2))/math.sqrt(self.embedding_dim)
        probs = self.softmax(scores)

        E = torch.matmul(probs, V)

        E = self.E_w(E)
        E = E + X_end

        # identity = self.downsample(E)
        out = E.squeeze(1)
        # identity = identity.squeeze(1)

        out = self.linear1(out)
        out = self.relu1(out)
        # out = self.linear2(out)
        # out = out + identity
        # out = self.relu2(out)

        out = self.linear3(out)

        return out

## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape


torch.Size([1, 29794])

In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3834752


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30074
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [18]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=512)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29067.98, valid ppl: 28321.16
10000 steps; 5120000 examples so far; train ppl: 629.79, valid ppl: 379.95
20000 steps; 10240000 examples so far; train ppl: 317.63, valid ppl: 280.73
30000 steps; 15360000 examples so far; train ppl: 259.40, valid ppl: 245.46
40000 steps; 20480000 examples so far; train ppl: 236.11, valid ppl: 227.57
50000 steps; 25600000 examples so far; train ppl: 221.79, valid ppl: 217.68
60000 steps; 30720000 examples so far; train ppl: 205.78, valid ppl: 211.25
70000 steps; 35840000 examples so far; train ppl: 198.29, valid ppl: 207.45
80000 steps; 40960000 examples so far; train ppl: 196.59, valid ppl: 203.15
90000 steps; 46080000 examples so far; train ppl: 194.90, valid ppl: 200.05
100000 steps; 51200000 examples so far; train ppl: 192.93, valid ppl: 197.45
110000 steps; 56320000 examples so far; train ppl: 188.81, valid ppl: 196.00
120000 steps; 61440000 examples so far; train ppl: 178.76, valid ppl: 194.74
130000 steps; 665

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [19]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 172.5462837789276


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz.
Eu gosto de comer pizza pois me faz..
Eu gosto de comer pizza pois me faz...
Eu gosto de comer pizza pois me faz....
Eu gosto de comer pizza pois me faz.....
Eu gosto de comer pizza pois me faz......
Eu gosto de comer pizza pois me faz.......
Eu gosto de comer pizza pois me faz........
Eu gosto de comer pizza pois me faz.........
Eu gosto de comer pizza pois me faz..........
Eu gosto de comer pizza pois me faz...........
Eu gosto de comer pizza pois me faz............
Eu gosto de comer pizza pois me faz.............
Eu gosto de comer pizza pois me faz..............
Eu gosto de comer pizza pois me faz...............
Eu gosto de comer pizza pois me faz................
Eu gosto de comer pizza pois me faz.................
Eu gosto de comer pizza pois me faz..................
Eu gosto de comer pizza pois me faz...................
Eu gosto de comer pizza pois me faz....................


In [24]:
prompt = 'Eu gosto de comer pizza do norte do'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza do norte do Brasil
Eu gosto de comer pizza do norte do Brasil.
Eu gosto de comer pizza do norte do Brasil. O
Eu gosto de comer pizza do norte do Brasil. O que
Eu gosto de comer pizza do norte do Brasil. O que é
Eu gosto de comer pizza do norte do Brasil. O que é o
Eu gosto de comer pizza do norte do Brasil. O que é o que
Eu gosto de comer pizza do norte do Brasil. O que é o que é
Eu gosto de comer pizza do norte do Brasil. O que é o que é o
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é o
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é o que
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é o que é
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é o que é o
Eu gosto de comer pizza do norte do Brasil. O que é o que é o que é o que é o que
Eu gosto de comer

In [29]:
prompt = 'Os dias são mais bonitos perto de casa da'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Os dias são mais bonitos perto de casa da minha
Os dias são mais bonitos perto de casa da minha vida
Os dias são mais bonitos perto de casa da minha vida.
Os dias são mais bonitos perto de casa da minha vida. O
Os dias são mais bonitos perto de casa da minha vida. O que
Os dias são mais bonitos perto de casa da minha vida. O que é
Os dias são mais bonitos perto de casa da minha vida. O que é o
Os dias são mais bonitos perto de casa da minha vida. O que é o que
Os dias são mais bonitos perto de casa da minha vida. O que é o que é
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o que
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o que é
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o que é o
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o que é o que
Os dias são mais bonitos perto de casa da minha vida. O que é o que é o que

In [33]:
prompt = 'Praticar exercícios todos os dias melhora o  '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Praticar exercícios todos os dias melhora o que
Praticar exercícios todos os dias melhora o que se
Praticar exercícios todos os dias melhora o que se refere
Praticar exercícios todos os dias melhora o que se refere a
Praticar exercícios todos os dias melhora o que se refere a sua
Praticar exercícios todos os dias melhora o que se refere a sua vida
Praticar exercícios todos os dias melhora o que se refere a sua vida.
Praticar exercícios todos os dias melhora o que se refere a sua vida. O
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que é
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que é o
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que é o que
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que é o que é
Praticar exercícios todos os dias melhora o que se refere a sua vida. O que é o que é o
Praticar exercíc